# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
import geoviews as gv
import pyproj
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,georgiyevka,43.0369,74.7114,23.04,63,0,2.24,KZ,1669663624
1,1,mataura,-46.1927,168.8643,52.32,80,100,1.81,NZ,1669663501
2,2,ushuaia,-54.8000,-68.3000,67.66,30,75,12.66,AR,1669663502
3,3,atuona,-9.8000,-139.0333,77.83,73,12,14.72,PF,1669663506
4,4,minsk,53.9000,27.5667,28.15,98,100,5.28,BY,1669663294
...,...,...,...,...,...,...,...,...,...,...
547,547,abu dhabi,24.4667,54.3667,80.62,74,0,3.44,AE,1669663389
548,548,havelock,34.8791,-76.9013,68.95,58,61,8.01,US,1669663679
549,549,visby,57.6409,18.2960,38.05,93,100,9.22,SE,1669663679
550,550,morris,40.8334,-74.5329,49.82,68,75,11.50,US,1669663679


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
# Configure the map plot

humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    title = "Humidity"
)


# Display the map plot
humidity_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down with ideal weather condition 
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] >= 65) &
                                   (city_data_df["Max Temp"] <= 80) &
                                   (city_data_df["Wind Speed"] < 10) &
                                   (city_data_df["Cloudiness"] <= 0)]

# Rearrange headers for DataFrame 
ideal_df = ideal_df[['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Date']]

ideal_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
16,cape town,ZA,-33.9258,18.4232,73.90,65,0,6.91,1669663514
98,kapaa,US,22.0752,-159.3190,76.98,95,0,5.75,1669663505
101,san patricio,US,28.0170,-97.5169,72.32,64,0,4.56,1669663517
185,canyon lake,US,29.8752,-98.2625,74.41,49,0,3.00,1669663466
188,bandrele,YT,-12.9067,45.1914,78.55,78,0,4.61,1669663641
269,esna,EG,25.2934,32.5540,68.02,29,0,5.57,1669663648
293,salalah,OM,17.0151,54.0924,75.29,78,0,2.30,1669663538
310,north myrtle beach,US,33.8160,-78.6800,71.11,56,0,8.05,1669663652
325,saint-pierre,RE,-21.3393,55.4781,74.88,64,0,6.91,1669663276
496,ixtapa,MX,20.7000,-105.2000,78.03,57,0,6.91,1669663570


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
hotel_df

# Display sample data
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
16,cape town,ZA,-33.9258,18.4232,73.90,65,0,6.91,1669663514,
98,kapaa,US,22.0752,-159.3190,76.98,95,0,5.75,1669663505,
101,san patricio,US,28.0170,-97.5169,72.32,64,0,4.56,1669663517,
185,canyon lake,US,29.8752,-98.2625,74.41,49,0,3.00,1669663466,
188,bandrele,YT,-12.9067,45.1914,78.55,78,0,4.61,1669663641,
269,esna,EG,25.2934,32.5540,68.02,29,0,5.57,1669663648,
293,salalah,OM,17.0151,54.0924,75.29,78,0,2.30,1669663538,
310,north myrtle beach,US,33.8160,-78.6800,71.11,56,0,8.05,1669663652,
325,saint-pierre,RE,-21.3393,55.4781,74.88,64,0,6.91,1669663276,
496,ixtapa,MX,20.7000,-105.2000,78.03,57,0,6.91,1669663570,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "categories":"accommodation.hotel",
          "apiKey":geoapify_key,
          
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng= row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: Townhouse Hotel
kapaa - nearest hotel: Pono Kai Resort
san patricio - nearest hotel: No hotel found
canyon lake - nearest hotel: Horseshoe Riverside Lodge
bandrele - nearest hotel: Les Baobabs
esna - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
north myrtle beach - nearest hotel: Mar Vista Grande
saint-pierre - nearest hotel: Tropic Hotel
ixtapa - nearest hotel: Ma. Cristina
new iberia - nearest hotel: Inn of Iberia
natchitoches - nearest hotel: College Inn


,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
16,cape town,ZA,-33.9258,18.4232,73.90,65,0,6.91,1669663514,Townhouse Hotel
98,kapaa,US,22.0752,-159.3190,76.98,95,0,5.75,1669663505,Pono Kai Resort
101,san patricio,US,28.0170,-97.5169,72.32,64,0,4.56,1669663517,No hotel found
185,canyon lake,US,29.8752,-98.2625,74.41,49,0,3.00,1669663466,Horseshoe Riverside Lodge
188,bandrele,YT,-12.9067,45.1914,78.55,78,0,4.61,1669663641,Les Baobabs
269,esna,EG,25.2934,32.5540,68.02,29,0,5.57,1669663648,No hotel found
293,salalah,OM,17.0151,54.0924,75.29,78,0,2.30,1669663538,Muscat International Hotel
310,north myrtle beach,US,33.8160,-78.6800,71.11,56,0,8.05,1669663652,Mar Vista Grande
325,saint-pierre,RE,-21.3393,55.4781,74.88,64,0,6.91,1669663276,Tropic Hotel
496,ixtapa,MX,20.7000,-105.2000,78.03,57,0,6.91,1669663570,Ma. Cristina


In [22]:
#Remove columns with no hotel found 
hotel_df_1 = hotel_df.drop(hotel_df.index[2])
hotel_df_1 = hotel_df_1.drop(hotel_df_1.index[4])
hotel_df_1

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
16,cape town,ZA,-33.9258,18.4232,73.90,65,0,6.91,1669663514,Townhouse Hotel
98,kapaa,US,22.0752,-159.3190,76.98,95,0,5.75,1669663505,Pono Kai Resort
185,canyon lake,US,29.8752,-98.2625,74.41,49,0,3.00,1669663466,Horseshoe Riverside Lodge
188,bandrele,YT,-12.9067,45.1914,78.55,78,0,4.61,1669663641,Les Baobabs
293,salalah,OM,17.0151,54.0924,75.29,78,0,2.30,1669663538,Muscat International Hotel
310,north myrtle beach,US,33.8160,-78.6800,71.11,56,0,8.05,1669663652,Mar Vista Grande
325,saint-pierre,RE,-21.3393,55.4781,74.88,64,0,6.91,1669663276,Tropic Hotel
496,ixtapa,MX,20.7000,-105.2000,78.03,57,0,6.91,1669663570,Ma. Cristina
523,new iberia,US,30.0035,-91.8187,74.97,40,0,4.61,1669663675,Inn of Iberia
529,natchitoches,US,31.7668,-93.1335,72.45,46,0,4.61,1669663675,College Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
# Configure the map plot

hotel_plot = hotel_df_1.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["Humidity", "Hotel Name", "Country"],
    geo = True,
    tiles = "OSM",
    size = "Population",
    color = "City"
)

# Display the map plot
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)